<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/042_ActionClass_ActionRegistry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A **blog writing agent pipeline** is a perfect hands-on project to reinforce the modular agent design and tool calling pattern. Let’s design a simplified **multi-stage agent** using mock functions and walk through it.

---

## ✍️ Blog Writing Agent — Idea

**Goal**: Generate, research, write, and refine a blog post with minimal user input.

---

### 🔧 Step 1: Define Actions (Tools)

Here are five mock actions we can implement:

| Tool Name            | Purpose                                |
| -------------------- | -------------------------------------- |
| `generate_blog_idea` | Proposes a blog idea on a given topic  |
| `research_topic`     | Returns mock research notes            |
| `write_draft`        | Writes a first draft of the blog post  |
| `review_draft`       | Returns a mock critique                |
| `rewrite_blog_post`  | Produces a revised version of the blog |

---

### 🧠 Step 2: Define the Mock Functions

```python
def generate_blog_idea(topic: str) -> str:
    return f"How {topic} is Changing the Future of Work"

def research_topic(topic: str) -> str:
    return f"Here are some stats, quotes, and arguments about {topic}..."

def write_draft(idea: str, research: str) -> str:
    return f"Draft blog post based on idea: {idea}\n\nSupported by: {research}"

def review_draft(draft: str) -> str:
    return "Your draft is strong, but the intro is weak and the tone is too formal."

def rewrite_blog_post(draft: str, feedback: str) -> str:
    return f"Final blog post with improved intro and friendlier tone."
```

---

### 🧱 Step 3: Register Actions

Each of these becomes an `Action`:

```python
action_registry.register(Action(
    name="generate_blog_idea",
    function=generate_blog_idea,
    description="Generate a blog title for a given topic.",
    parameters={
        "type": "object",
        "properties": {
            "topic": {"type": "string"}
        },
        "required": ["topic"]
    }
))

# ... Repeat for other 4 actions
```

---

### 🎯 Step 4: Define Goals

```python
goals = [
    Goal(priority=1, name="Ideate", description="Come up with a good blog topic."),
    Goal(priority=2, name="Research", description="Gather useful info about the topic."),
    Goal(priority=3, name="Draft", description="Write the initial blog post."),
    Goal(priority=4, name="Review", description="Critique and improve the writing."),
    Goal(priority=5, name="Rewrite", description="Produce the final version."),
    Goal(priority=6, name="Terminate", description="End with a finished blog post.")
]
```

---

### 💻 Step 5: Simulate User Input

```python
user_input = "I want to write a blog about AI in Education."
```

---

### 🔁 Agent Loop (Simplified)

The agent will:

1. Use `generate_blog_idea` with topic: `"AI in Education"`
2. Then call `research_topic` with the same topic
3. Call `write_draft` with the idea and research
4. Review it with `review_draft`
5. Use `rewrite_blog_post` with draft and review
6. Terminate and return the final post

---

### ✅ Benefits of this Practice Agent

* You learn how to **chain actions** with arguments
* Shows how **memory** stores intermediate outputs
* Clear GOAL → TOOL → RESULT loop
* Builds confidence before handling real API tools






### 🧠 Agent Capabilities:

1. **Generate Blog Idea** – from user-provided topic
2. **Research Topic** – pulls mock background info
3. **Write Draft** – based on idea + research
4. **Review Draft** – provides editorial feedback
5. **Rewrite Post** – improves draft based on feedback
6. **Terminate** – closes loop and returns final post

### 🧱 Technical Features:

* Clear `Action` class structure with metadata + validation
* Fully registered with an `ActionRegistry`
* Simulated loop walks through entire content lifecycle






In [ ]:
from typing import List, Callable
from dataclasses import dataclass
import json

# === 1. Define Core Classes ===
class Action:
    def __init__(self, name: str, function: Callable, description: str, parameters: dict, terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.parameters = parameters
        self.terminal = terminal

class ActionRegistry:
    def __init__(self):
        self.actions = {}

    def register(self, action: Action):
        self.actions[action.name] = action

    def get_action(self, name: str):
        return self.actions.get(name)

    def get_actions(self):
        return list(self.actions.values())

@dataclass(frozen=True)
class Goal:
    priority: int
    name: str
    description: str

# === 2. Mock Functions ===
def generate_blog_idea(topic: str) -> str:
    return f"How {topic} is Changing the Future of Work"

def research_topic(topic: str) -> str:
    return f"Here are some stats, quotes, and arguments about {topic}..."

def write_draft(idea: str, research: str) -> str:
    return f"Draft based on: {idea}\n\nSupported by research: {research}"

def review_draft(draft: str) -> str:
    return "Intro needs more punch. Tone could be friendlier."

def rewrite_blog_post(draft: str, feedback: str) -> str:
    return "Final blog post with improved intro and casual tone."

def terminate(message: str) -> str:
    return message

# === 3. Register Actions ===
action_registry = ActionRegistry()

action_registry.register(Action(
    name="generate_blog_idea",
    function=generate_blog_idea,
    description="Generate a blog idea from a topic.",
    parameters={"type": "object", "properties": {"topic": {"type": "string"}}, "required": ["topic"]},
))

action_registry.register(Action(
    name="research_topic",
    function=research_topic,
    description="Research a given blog topic.",
    parameters={"type": "object", "properties": {"topic": {"type": "string"}}, "required": ["topic"]},
))

action_registry.register(Action(
    name="write_draft",
    function=write_draft,
    description="Create a draft blog post.",
    parameters={"type": "object", "properties": {"idea": {"type": "string"}, "research": {"type": "string"}}, "required": ["idea", "research"]},
))

action_registry.register(Action(
    name="review_draft",
    function=review_draft,
    description="Review the blog post and give feedback.",
    parameters={"type": "object", "properties": {"draft": {"type": "string"}}, "required": ["draft"]},
))

action_registry.register(Action(
    name="rewrite_blog_post",
    function=rewrite_blog_post,
    description="Rewrite blog post using feedback.",
    parameters={"type": "object", "properties": {"draft": {"type": "string"}, "feedback": {"type": "string"}}, "required": ["draft", "feedback"]},
))

action_registry.register(Action(
    name="terminate",
    function=terminate,
    description="Ends the loop and outputs the blog post.",
    parameters={"type": "object", "properties": {"message": {"type": "string"}}, "required": ["message"]},
    terminal=True
))

# === 4. Simulated Agent Loop ===
user_topic = "AI in Education"
print(f"\nUser wants to write about: {user_topic}\n")

idea = generate_blog_idea(user_topic)
print(f"👉 Idea: {idea}")

research = research_topic(user_topic)
print(f"📚 Research: {research}")

draft = write_draft(idea, research)
print(f"📝 Draft: {draft}")

feedback = review_draft(draft)
print(f"🧠 Feedback: {feedback}")

final_blog = rewrite_blog_post(draft, feedback)
print(f"✅ Final Blog: {final_blog}\n")

summary = terminate(f"Blog created: {idea}\n\n{final_blog}")
print(f"🎉 Termination Message: {summary}")


```python
from typing import List, Callable
```

These two imports are from Python's `typing` module, and they’re used for **type hinting**—a way to tell Python (and your editor or tools like linters and type checkers) what kind of values are expected.

---

### 🔹 `from typing import List`

* This tells Python that when you write `List[str]`, it means:
  “A list where every element is a string.”

**Example:**

```python
def list_python_files() -> List[str]:
    return ["main.py", "utils.py"]
```

⬆️ That tells the reader (and tools like `mypy` or your IDE) that the function returns a list of strings.

---

### 🔹 `from typing import Callable`

* This represents **a function** (a callable object). You use this when you want to indicate that a variable holds a function.

**Example:**

```python
def run_tool(tool: Callable):
    return tool()
```

⬆️ This says: "`tool` is a function I can call, and I’m going to call it."

You can be even more specific:

```python
Callable[[str], int]
```

That would mean: a function that takes a `str` and returns an `int`.

---

### ✅ Why it matters for agents

When you define your `Action` class, you’ll often see:

```python
class Action:
    def __init__(self, name: str, function: Callable, ...)
```

That’s saying:

* `name`: should be a string
* `function`: should be something you can call (like a Python function)

This makes the code easier to read, safer to work with, and IDEs will give you smarter autocomplete + error checking.




## 🔹 What Is a `Callable`?

A **`callable`** is *anything* in Python that you can "call" like a function using parentheses:

```python
result = something()
```

If `something()` works, then `something` is a **callable**.

---

## ✅ Examples of Callables

### 1. **Functions**

```python
def greet():
    return "Hello"

greet()  # ✅ This works → it's a callable
```

### 2. **Built-in functions**

```python
len("hello")  # ✅ len is a callable
```

### 3. **Lambda functions**

```python
add = lambda x, y: x + y
add(2, 3)  # ✅ Callable
```

### 4. **Classes (because they can be instantiated)**

```python
class Dog:
    def __init__(self):
        print("Bark!")

Dog()  # ✅ This works → Dog is callable
```

### 5. **Objects with `__call__` defined**

```python
class Greeter:
    def __call__(self):
        return "Hi!"

g = Greeter()
g()  # ✅ This works → object is callable
```

---

## 🧠 Why Use `Callable` in Type Hints?

If you want to accept a function as an argument, you use `Callable`:

```python
from typing import Callable

def run_tool(tool: Callable):
    return tool()
```

This says: "`tool` must be something I can call—like a function."

You can also specify *input and output types*:

```python
Callable[[int, int], int]  # function that takes 2 ints and returns an int
```

---

## Summary

| Thing             | Is it Callable? | Why?                    |
| ----------------- | --------------- | ----------------------- |
| `def func()`      | ✅               | It’s a function         |
| `lambda x: x + 1` | ✅               | It’s a lambda function  |
| `len`             | ✅               | Built-in function       |
| `Dog()`           | ✅               | It instantiates a class |
| `"hello"`         | ❌               | Strings aren't callable |



You can think of `Callable` as a **type-aware placeholder for a function**. Here's what makes it powerful:

---

### 🔧 **Why `Callable` Is More Than Just a Placeholder**

1. **📍 Placeholder for a Function**
   It's like saying:

   > "Hey, this argument must be something you can call later (like `my_func()`), not just data like a string or number."

2. **✅ Type Checking**
   With tools like `mypy` or VSCode's intellisense, Python can **warn you early** if you accidentally pass something that isn't a function.

   ```python
   def my_tool(): pass

   Action("my_tool", my_tool, ...)     # ✅ OK
   Action("bad_tool", "not a function", ...)  # ❌ Type checker would flag this
   ```

3. **🧠 Self-Documentation**
   It communicates to future readers (and to you!) that this value isn't just for storing — it's meant to be **executed later**.

4. **🧱 Flexible With Parameters**
   You can even specify what kind of function it is, like:

   ```python
   Callable[[str], str]  # function that takes a str and returns a str
   Callable[[], None]    # function that takes nothing and returns nothing
   ```

---

### 🧠 Analogy: A Callable Is Like...

A **"promise of behavior"**:

> “I don’t know what this tool does yet, but I guarantee you can run it when you're ready.”

That’s perfect for agent design, because the agent may not run the function **immediately**—it stores it, then decides to invoke it later based on the LLM’s response.

---

Would you like to try writing a simple `Action` using a custom `Callable` to practice?


---

If you're looking at this `Action` class:

```python
from typing import Callable

class Action:
    def __init__(self, name: str, function: Callable, description: str, parameters: dict, terminal: bool = False):
        self.name = name
        self.function = function
        self.description = description
        self.parameters = parameters
        self.terminal = terminal
```

Let's focus on **why** we use `Callable` here for the `function` argument.

---

### ✅ What It Means

`function: Callable` says:

> “This `function` argument must be something that can be **called**, like a Python function (or anything else callable).”

So when you write:

```python
def read_file(file_name):
    return "some contents"

action = Action(
    name="read_file",
    function=read_file,  # 👈 this is the function being passed in
    description="Reads a file",
    parameters={"type": "object", "properties": {"file_name": {"type": "string"}}, "required": ["file_name"]}
)
```

You're passing the **function object**, not the result of calling it.

---

### 🤔 Why Is `Callable` Useful?

1. **Type Safety** – It lets type checkers (like in VSCode or with `mypy`) catch mistakes if you pass something that *isn't* a function.

2. **Clarity** – You're documenting that this class isn't just holding strings or data—it's holding **code logic** that can be run later.

3. **Flexibility** – The agent doesn’t run the tool immediately; it stores the reference to the tool function. Later, it can run:

   ```python
   result = action.function(**args)
   ```

---

### 🧠 Without `Callable`, This Would Still Work...

…but you’d lose:

* Type hints and IDE help
* Protection against passing non-functions (like strings or numbers)
* Clear self-documentation that this is an *executable function*



The `ActionRegistry` is a simple but powerful class that serves as a **central lookup table** or **directory** for all available actions (tools) your agent can use.

Let’s walk through each part:

---

## 🔁 `class ActionRegistry`

This is your **toolbox manager** — it stores all your `Action` objects and provides easy access to them.

---

### 🧱 `__init__(self)`

```python
def __init__(self):
    self.actions = {}
```

* Initializes an **empty dictionary** where:

  * The **keys** are the action names (strings),
  * The **values** are `Action` instances.
* Think of it as:

  ```python
  {
    "list_files": <Action object>,
    "read_file": <Action object>,
    ...
  }
  ```

---

### ➕ `register(self, action: Action)`

```python
def register(self, action: Action):
    self.actions[action.name] = action
```

* Adds a new `Action` to the registry.
* `action.name` is used as the key so the agent can later **refer to actions by name** when making a tool call.

📌 Example:

```python
registry = ActionRegistry()
registry.register(Action("read_file", read_file, "Reads a file", {...}))
```

---

### 🔍 `get_action(self, name: str)`

```python
def get_action(self, name: str):
    return self.actions.get(name)
```

* Retrieves an `Action` by its name (string).
* Used when the **LLM says "call read\_file"**, and you need to:

  1. Look it up,
  2. Get its function and schema,
  3. Call it with arguments.

---

### 📋 `get_actions(self)`

```python
def get_actions(self):
    return list(self.actions.values())
```

* Returns **all registered `Action` instances** as a list.
* Typically used when building the prompt — so the LLM knows **which tools are available**, including their names, descriptions, and parameters.

📌 Example output:

```python
[
  Action(name="list_files", ...),
  Action(name="read_file", ...),
  ...
]
```

---

## ✅ Why It’s Useful

* 📦 Centralized management of tools
* 📚 Easy to introspect (for building prompts)
* 🔐 Prevents naming conflicts
* 🧩 Pluggable: you can register/unregister actions dynamically
* 💡 Swappable: pass it into different agents for different tasks






### ✅ With `Action` and `ActionRegistry` (Modular Approach):

| Benefit             | How It Helps                                                                                                                                  |
| ------------------- | --------------------------------------------------------------------------------------------------------------------------------------------- |
| **Scalability**     | Adding a new action is as simple as defining a new function and registering it with a descriptive schema. No need to modify core agent logic. |
| **Maintainability** | Each action is self-contained with its own schema and metadata, so debugging a failing tool is localized to that tool.                        |
| **Readability**     | Clear separation of roles: actions do work, the registry manages them, and the agent executes based on goals.                                 |
| **Extensibility**   | Want to log every tool call, or check performance? You can wrap this in `Action.function()` without touching agent logic.                     |
| **Testability**     | You can test individual actions (functions) in isolation, or even mock them in tests without altering your registry.                          |

---

### 🧪 With Traditional (Hardcoded) Approach:

| Limitation                  | Why It Hurts                                                                                           |
| --------------------------- | ------------------------------------------------------------------------------------------------------ |
| **Tool spaghetti**          | You may end up with a massive `if tool_name == "X"` routing block. Easy to break, hard to debug.       |
| **Code duplication**        | Adding or reusing tools across projects often requires copy-pasting large chunks of code.              |
| **Tight coupling**          | Tool logic is intertwined with routing and agent behavior. Changing one means touching multiple areas. |
| **Harder to track changes** | No central registry means no single place to see all the tools your agent has access to.               |

---

### 🔁 Summary

With the modular architecture using `Action` and `ActionRegistry`, you're essentially building a **plugin system for your agent** — one that:

* abstracts tool complexity,
* centralizes control,
* and scales without losing structure.

This is a *huge advantage* when your agent grows from 3 tools to 30 or 300 — and keeps you from turning your project into an unmanageable mess.


